# Parsing file with backtesting result

In [57]:
import pickle

In [58]:
result_path = 'results/bollinger_rsi60min70_30/result.pickle'
#result_path = 'results/bollinger_rsi60min80_20/result.pickle'
#result_path = 'results/rsi60min80_20/result.pickle'

In [59]:
with open(result_path, 'rb') as file:
    backtesting_result = pickle.load(file)

In [60]:
backtesting_result = [byte_string.decode("utf-8") for byte_string in backtesting_result]

In [26]:
len(backtesting_result)

310

#### Calculating the whole financial result across all assets

In [43]:
# '--' is added as the separator to extract ticker and final financial result
end_fin_results = [string.split('--') for string in backtesting_result]

In [52]:
# Only the second element has the needed result
bt_result = []
for result in end_fin_results:
    try:
        bt_result.append(result[1])
    except Exception as e:
        continue

In [45]:
len(bt_result)

239

In [53]:
# Forms a dict from the string of format 'ticker financial_result' and filtes it
bt_result = {
    result.split(' ')[0]: float(result.split(' ')[1]) for result in bt_result
    #if float(result.split(' ')[1]) > 100000
}

In [54]:
len(bt_result)

239

In [55]:
bt_result = dict(sorted(bt_result.items(), key=lambda item: item[1], reverse=True))

In [34]:
sum([float(fin_result) - 100000 for fin_result in bt_result.values()])

1613121.0756625205

In [81]:
bt_result

{'RNFT': 147694.25763999938,
 'RLMNP': 129352.22633200024,
 'APTK': 126006.87571580059,
 'KMAZ': 123317.43338999979,
 'FIVE': 123278.60500000045,
 'SVAV': 122394.42940000004,
 'ENRU': 121687.70059774986,
 'AGRO': 121600.2955,
 'MOBB': 121534.8307999997,
 'MGTS': 120881.67399999991,
 'RAVN': 119655.65076999996,
 'LSNG': 118952.53138999981,
 'UPRO': 118756.9265524005,
 'OGKB': 118738.92019347977,
 'MRKU': 118736.06512480004,
 'MTSS': 118088.67484000031,
 'GTRK': 115399.75888000055,
 'URKZ': 115145.576,
 'TGKDP': 114645.66618337749,
 'RBCM': 114594.95418760015,
 'DZRDP': 114370.84600000014,
 'OBUV': 114353.53790000001,
 'BANE': 114296.80640000006,
 'KAZT': 114234.01391999998,
 'RUAL': 114154.93202200026,
 'VLHZ': 114137.51415499992,
 'KUZB': 114094.383109936,
 'UWGN': 113949.28800999997,
 'MRKK': 113811.95843200014,
 'BSPB': 113698.8774649997,
 'UNAC': 113330.23825600027,
 'GCHE': 113296.59340000025,
 'LSRG': 113177.92234000043,
 'KBSB': 112911.36144999992,
 'MRKV': 112897.39012465115,
 '

In [28]:
'''with open('PROFITABLE_TICKERS.pickle', 'wb') as file:
    pickle.dump(list(bt_result.keys()), file)'''

#### Analysis of the net results distribution for every assets

In [61]:
import re

In [72]:
dict_of_net_results = {}
for result in backtesting_result:
    try:
        # Splits ticker extraction on two steps for readability
        ticker_and_result = result.split('--')[1]
        ticker = ticker_and_result.split(' ')[0]
                       
        list_of_net_results = re.findall(r'NET (.*)', result)
        list_of_net_results = list(map(float, list_of_net_results))
        
        dict_of_net_results[ticker] = list_of_net_results
    except Exception as e:
        pass

In [82]:
len(dict_of_net_results)

239

In [88]:
(dict_of_net_results['RNFT'])

NameError: name 'mean' is not defined

# Test of correctness of indicators

In [1]:
import pandas as pd
from ta.momentum import RSIIndicator

In [2]:
prices = pd.read_csv('/mnt/tick_prices/SBER.csv')

In [4]:
prices.tail()

,datetime,date,time,last
31709941,1970-01-01 00:00:01.619117377,1970-01-01,00:00:01.619117,292.21
31709942,1970-01-01 00:00:01.619117377,1970-01-01,00:00:01.619117,292.21
31709943,1970-01-01 00:00:01.619117391,1970-01-01,00:00:01.619117,292.21
31709944,1970-01-01 00:00:01.619117391,1970-01-01,00:00:01.619117,292.21
31709945,1970-01-01 00:00:01.619117391,1970-01-01,00:00:01.619117,292.21


In [11]:
prices['datetime'] = pd.to_datetime(prices['datetime'])

In [13]:
prices.set_index('datetime', inplace=True)

In [15]:
prices_resampled_60min = prices['last'].resample('60Min').ohlc()

In [17]:
prices_resampled_60min.tail()

,open,high,low,close
datetime,,,,
2021-04-22 14:00:00,287.52,293.15,287.32,292.39
2021-04-22 15:00:00,292.40,293.60,291.61,292.89
2021-04-22 16:00:00,292.92,294.67,292.51,293.69
2021-04-22 17:00:00,293.63,293.87,292.36,293.28
2021-04-22 18:00:00,293.27,293.42,292.00,292.21


In [19]:
rsi = RSIIndicator(close=prices_resampled_60min.close)

In [26]:
rsi.rsi().tail(50)

datetime
2021-04-20 17:00:00    33.830679
2021-04-20 18:00:00    43.758962
2021-04-20 19:00:00    41.266837
2021-04-20 20:00:00    43.960706
2021-04-20 21:00:00    43.296993
2021-04-20 22:00:00    44.401862
2021-04-20 23:00:00    40.963540
2021-04-21 00:00:00    40.963540
2021-04-21 01:00:00    40.963540
2021-04-21 02:00:00    40.963540
2021-04-21 03:00:00    40.963540
2021-04-21 04:00:00    40.963540
2021-04-21 05:00:00    40.963540
2021-04-21 06:00:00    40.963540
2021-04-21 07:00:00    40.963540
2021-04-21 08:00:00    40.963540
2021-04-21 09:00:00    40.963540
2021-04-21 10:00:00    67.759598
2021-04-21 11:00:00    51.683164
2021-04-21 12:00:00    42.531836
2021-04-21 13:00:00    55.954915
2021-04-21 14:00:00    52.281464
2021-04-21 15:00:00    48.130044
2021-04-21 16:00:00    47.464657
2021-04-21 17:00:00    49.494871
2021-04-21 18:00:00    46.620467
2021-04-21 19:00:00    49.314924
2021-04-21 20:00:00    50.269060
2021-04-21 21:00:00    50.269060
2021-04-21 22:00:00    56.518611
2